# Valutazione del Modello
Sommario per metriche  che si utilizzano per valutare un modello:

<img src="https://drive.google.com/uc?id=184NgFeVKI2uLD5eyKessu4TmopKk-q9w">

## Accuratezza:

É la metrica più comune e rappresenta la frazione dei campioni predetti correttamente:

$$
accuracy(y_i, \hat{y_i}) = \frac{1}{n_{samples}} \sum_{i=0}^{n_{samples}-1} 1(y_i = \hat{y_i})
$$

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = [0, 2, 1, 3]
y_true = [0, 1, 2, 3]
print(accuracy_score(y_true, y_pred)) # y vero rispetto a quello predetto
print(accuracy_score(y_true, y_pred, normalize=False)) #  numero di istanze che non sono corrette

Ma l’accuratezza non é tutto , si deve tenere conto anche in certi casi della precision e della recall anche essere avranno la loro metrica

In [ ]:
from sklearn.metrics import precision_score
y_pred = [0, 1, 1, 0]
y_true = [0, 1, 0, 1]
print(precision_score(y_true, y_pred))

# Metriche per la regressione:

In modelli di regressione non si può utilizzare l’accuratezza,  é necessario utilizzare altre metriche “matematiche” per poter calcolare la precisione di un modello nel predire correttamente un numero, quello che succede si solito é una misura della differenza tra il valore aspettato ( 100K euro di casa ) e il valore predetto ( 170K euro di casa predetta ).

Ci sono diversi tipi di metriche e tutte si trovano importandole da sklearn, la differenza é poco importante per il corso in questione:

- `explained_variance_score`
- `max_error`
- `mean_absolute_error`
- `mean_squared_error`
- `Root_squared_error`
- `mean_squared_log_error`
- `median_absolute_error`
- `r2_score`

Di questa lista  una nota importante c’é l’ha il  **Root_squared_error(RMSE)**:

É la metrica più utilizzata nei modelli regressivi e   rappresenta la radice quadrata della media dei valori quadrati:

$$
 RMSE(y, \hat{y}) = \sqrt{\frac{1}{n_{samples}} \sum_{i=0}^{n_{samples}} (y_{i}-\hat{y_{i}})^2}
$$

e viene implementata in questo modo:

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
sqrt(mean_squared_error(y_true, y_pred))

# Cross Validation:

la cross validation testa il modello **k**  dividendo il dataset in **k** parti.
Se prendiamo per esempio un modello diviso e allenato con il `train_test_split`:

In [ ]:
from sklearn.datasets import load_wine as wine
from sklearn.model_selection import train_test_split
from sklearn import svm
wines = wine()

X_train, X_test, y_train, y_test = train_test_split(
    wines.data, wines.target, test_size=0.4, random_state=0)

model = svm.SVC(kernel="linear",C=1).fit(X_train,y_train)
model.score(X_test,y_test)

Differenti iperparametri per gli estimatori, come per esempio il `C` per il `svc` hanno comunque   un forte rischio di fare overfitting sul test perché i parametri possono modificarsi finché l’estimatore non performa in modo ottimo.

La conosce del test può essere perduta.

per questo si utilizzano le **CV:**

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model,wines.data,wines.target, cv=5)
scores

In [ ]:
scores.mean()

## Cross  validate:

la `cross_validate` si differenzia con la `cross_val_score` per due motivi:

1. Permette di specificare più metriche per la stessa valutazione
2. ritorna un dict che contiene i tempi di fit e dello score, oltre al test score

In [ ]:
from sklearn.model_selection import cross_validate
scoring = ['precision_macro', 'recall_macro','accuracy']
scores = cross_validate(model, wines.data, wines.target, scoring= scoring,cv=5)
sorted(scores.keys())

In [ ]:
scores['test_precision_macro']

In [ ]:
scores['fit_time']

In [ ]:
scores

## Cross validation iterators

questa sezione non é obbligatoria ma é doveroso fare un piccolo sunto perché potrebbe comunque arrivare all’esame. Si tratta di una lista di strumenti che generano indici che possono essere utilizzati per dividere un dataset  a seconda di diverse strategie di cross validation.

Si possono assumere corrette queste tecniche di cross validation se possiamo assumere  che i dati sono indipendenti gli uni dagli altri e identicamente distribuiti (i.i.d)

- **K-fold**
- **Repeated K-fold**
- **Leave One Out(LOO)**
- **Leave P Out(LPO)**
- **Shuffle & Split**

# Dummy estimators:

Si tratta di un classificatore “*********stupido”********* che serve unicamente per fare il confronto:
il classificatore ha un parametro `strategy`:

- `"most_frequent"`: predice sempre con la classe più frequente osservata
- `“stratified”`: ritorna sempre la classe che ha probabilità 1 nel vettore one-hot nel metodo `predict_proba`
- `“uniform”`: genera le predizione in modo uniformemente random
- `“constant”`: predice sempre un unica label che viene fornita dall'utente

In [ ]:
from sklearn.dummy import DummyClassifier

print(model.score(X_test,y_test))

dummy_model = DummyClassifier(strategy="most_frequent",random_state=0)
dummy_model.fit(X_train,y_train)
result = dummy_model.predict(X_test)

accuracy_score(result,y_test)

# GridSearch:

Permette di organizzare batterie di test con dei parametri che noi selezioniamo:

In [ ]:
from sklearn.model_selection import GridSearchCV

# specifichiamo i parametri
params = {'kernel':('linear','rbf','poly'), 'C':[0.1,1,10]} # due parametri  -> farà 9 *  5 = 45 tester -> sono molti per numero molto grandi
model = svm.SVC(gamma="scale")
grid_model = GridSearchCV(model,params,cv=5)
grid_model.fit(wines.data,wines.target)

In [ ]:
grid_model.get_params() # dice i parametri utilizzati

In [ ]:
grid_model.cv_results_

In [ ]:
grid_model.best_estimator_

In [ ]:
grid_model.best_score_